In [1]:
import time
from bs4 import BeautifulSoup
from time import sleep
import requests
from random import randint
from html.parser import HTMLParser
import json

In [2]:
USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

In [38]:
import requests

class SearchEngine:
    
    @staticmethod
    def search(inputData,sleep=True):
        finalRes=[]
        for i in range(len(inputData)):
#             if sleep:
#                 time.sleep(randint(10,15))
            results = []
            response = "https://www.duckduckgo.com/html/?q="+inputData[i]
            soup = BeautifulSoup(requests.get(response, headers=USER_AGENT).text,"html.parser")
            results = SearchEngine.scrape_search_result(soup)
            finalRes.append(results[:10])
        return finalRes
    
    @staticmethod
    def scrape_search_result(soup):
        raw_results = soup.find_all("a", attrs = {"class" : "result__a"})
        results = []
        for result in raw_results:
            link = result.get('href')
            results.append(link)
        return results
    
    @staticmethod
    def read_file(fileName):
        inputData=[]
        with open(fileName, "r") as f:
            line = f.readline()
            while line:
                inputData.append(line.rstrip())
                line = f.readline()
        return inputData
    

In [39]:
inputQuery = SearchEngine.read_file("100QueriesSet4.txt")

In [40]:
final=SearchEngine.search(inputQuery)

In [41]:
data = {}
for query, result in zip(inputQuery, final):
    data[query] = result

In [42]:
with open("hw1_4.json", "w") as f:
    json.dump(data, f)

In [43]:
with open("Google_Result4.json", "r") as file:
    # Use json.load() to parse the JSON data
    data2 = json.load(file)
with open("hw1_4.json", "r") as file:
    # Use json.load() to parse the JSON data
    data = json.load(file)

In [49]:
n_res = []
spe_res = []
for k,v in data.items():
    arr1 = v
    arr2 = data2[k]
    set1 = set(arr1)
    set2 = set(arr2)
    overlap = set1.intersection(set2)
    n = len(overlap)
    dsq = 0
    spe = 2
    if len(overlap) == 0:
        spe = 0
    for j in overlap:
        if len(overlap)== 1:
            if arr1.index(j) != arr2.index(j):
                spe = 0
            else:
                spe = 1
        else:
            dsq += (arr1.index(j) - arr2.index(j))**2
    if spe == 2:
        spe = 1-((6*dsq)/(n*(n**2-1)))
    n_res.append(len(overlap))
    spe_res.append(spe)


In [46]:
import csv

# list of data

data_res = [['Queries', 'Number of Overlapping Results','Percent Overlap',"Spearman Coefficient"]]
count = 0
total_n = 0
total_n_per = 0
total_spe = 0
for k,v in data.items():
    res = [k,n_res[count],float(n_res[count]*10),spe_res[count]]
    total_n += n_res[count]
    total_n_per += float(n_res[count]*10)
    total_spe +=spe_res[count]
    data_res.append(res)
    count +=1
data_res.append(["Averages",total_n/100,total_n_per/100,total_spe/100])

In [47]:
filename = "hw1_4.csv"


with open(filename, 'w', newline='') as csvfile:

    csvwriter = csv.writer(csvfile)

    csvwriter.writerows(data_res)